In [1]:
import sys
sys.version_info

sys.version_info(major=3, minor=8, micro=10, releaselevel='final', serial=0)

In [11]:
import os, enum;
from abc import ABC, abstractmethod

class File:
    dir = None
    fileName = None
    fullyQualified = None
    def __init__(self, dir, fileName):
        self.dir = dir
        self.fileName = fileName
        self.fullyQualified = dir + '/' + fileName;
        
def addFiles(fileList, directory):
   for root, dirs, files in os.walk(directory):
        for file in files:
            fileList.append(File(root, file))
        for subdir in dirs:
            addFiles(fileList, subdir);
            
    
class Result(enum.Enum):
    failedMatch = 0           #didn't match - reset to zero
    matchedProgress = 1       #all requirements met - move on to the next requirement 
    oneOfManyMatches = 2      #Data gathered - keep on feeding me more lines
    endOfSet = 3              #contiguous set has ended - re-analyze this line
    gameSetMatch = 4          #Found end of data set - go spit out results
"""
    define an array of matching patterns
    Params:
    1 - Index to expect parameter 2
    2 - Space delimited match string
    3 - Data array index to capture
 - Populated value - starts as None (null)
"""
class Requirement(ABC):
    @abstractmethod
    def testline(self, line):
        pass
    @abstractmethod
    def reset(self):
        pass
    @abstractmethod
    def printResults(self, stream):
        pass
class Splicer(Requirement):
    def __init__(self, matchIdx, matchValue, captureIndex, foundValue):
        self.matchIdx = matchIdx
        self.matchValue = matchValue
        self.captureIndex = captureIndex
        self.foundValue = foundValue
    def printResults(self, stream):
        stream.write(self.matchValue+'\t'+self.foundValue+'\t')
    @abstractmethod
    def getSpliceChar(self):
        pass
    def testline(self, line):
        sp = line.split(self.getSpliceChar())
        if (sp is not None and 
            len(sp) > self.matchIdx and 
            len(sp) > self.captureIndex and 
            self.matchValue == sp[self.matchIdx]):
            self.foundValue = sp[self.captureIndex]
            return Result.matchedProgress
        return Result.failedMatch
    def reset(self):
        self.foundValue = None
        
class SpaceSplicer(Splicer):
    def getSpliceChar(self):
        return ' '

class ColonSplicer(Splicer): #Ouch?
    def getSpliceChar(self):
        return ':'
class Blank(Requirement):
    def reset(self):
        return;
    def printResults(self, stream):
        return;
    def testline(self, line):
        if (line is None):
            print('Not sure what to: do with None-zies')
        elif (len(line.strip()) == 0):
            #blank line!
            return Result.matchedProgress
        return Result.failedMatch
class Comment(Requirement):
    def reset(self):
        self.comment = None;
    def printResults(self, stream):
        return;
    def testline(self, line):
        self.comment = line.strip()
        return Result.matchedProgress
    
class FileInfo:
    def __init__(self, extension):
        self.extension = extension
        self.textLineCount = 0;
        self.binByteCount= 0;
        self.isBinary = False
        self.inserts = 0
        self.deletes = 0
class FileCommit(Requirement):
    def __init__(self):
        self.reset()
    def reset(self):
        self.extensionDic = {}
    def printResults(self, stream):
        for fc in self.extensionDic:
            stream.write('\n\text\t'+str(fc.extension)+'\t')
            stream.write('textLineCount\t'+str(fc.textLineCount)+'\t')
            stream.write('binByteCount\t'+str(fc.binByteCount)+'\t')
            stream.write('isBinary\t'+str(fc.isBinary)+'\t');
            stream.write('inserts\t'+str(fc.inserts)+'\t');
            stream.write('deletes\t'+str(fc.deletes)+'\t');
    def testline(self, line):
        sp = line.split('|')
        validData = False;
        if (len(sp) > 1):
            fileNameArray = sp[0].split('/');
            fileName = fileNameArray[len(fileNameArray)-1];            
            dotSplit = fileName.split('.')
            ext = None
            if (fileName.startswith('.') == False and len(dotSplit) > 1):
                ext = dotSplit[len(dotSplit)-1] #last element (e.g. '.txt')
            fi = self.extensionDic.get(ext)
            if (fi is None):
                fi = FileInfo(ext)
                self.extensionDic[ext] = fi
            size = sp[1].split(' ')
            if (size[0].isnumeric and len(size[0]) > 0):
                fi.isBinary = False
                fi.textLineCount = int(size[0])
                pc = 0
                mc = 0
                plus = size[1].split('+')
                for p in plus:
                    if (len(p) == 0):
                        fi.inserts += 1
                        pc += 1
                    else:
                        fi.deletes = len(p.split('-'))
                        mc += 1
                if (pc > 0 or lc > 0):
                    validData = True
            elif (size[0] == 'Bin'):
                sizeBefore = int(size[1]) if size[1].isnumeric() else -1
                sizeAfter = int(size[3]) if size[3].isnumeric() else -1
                if (sizeBefore >= 0 and sizeAfter >= 0):
                    validData = True
                    fi.isBinary = True
                    fi.binByteCount = sizeAfter - sizeBefore
        if (validData):
            return Result.oneOfManyMatches
        else:
            return Result.endOfSet
        
class Summary(Requirement):
    def reset(self):
        self.junk = ''
    def testline(self, line):
        print('Testing for Summary line ('+str(line)+')')
        sp = line.split(',')
        if (len(sp) < 2):
            print('ERROR - Summary line should have comma-separated change and insertion totals:'+line)
        else:
#442 files changed, 220745 insertions(+)
            words = sp[0].split(' ')
            if (len(words) > 0 and words[0].isnumeric()):
                self.filesChanged = int(words[0])
            words = sp[1].split(' ')
            if (len(words) > 0 and words[0].isnumeric()):
                self.insertionCount = int(words[0])
        return Result.gameSetMatch
    def printResults(self, stream):
        stream.write('ChangedFiles\t'+str(self.filesChanged)+'\tInsertionCound\t'+str(self.insertionCount)+'\t')
class RequirementSet:
    def __init__(self):
        self.reqArray = []
        self.reqArray.append( SpaceSplicer(0, 'commit', 1, None) )
        self.reqArray.append( ColonSplicer(0, 'Author', 1, None) )
        self.reqArray.append( ColonSplicer(0, 'Date', 1, None) )
        self.reqArray.append( Blank() )
        self.reqArray.append( Comment() )
        self.reqArray.append( Blank() )
        self.reqArray.append( FileCommit() )
        self.reqArray.append( Summary() )
        self.reqIndex = 0
        self.dataMatchesFound = 0
        self.indexErrorDic = {}
    def reset(self):
        self.reqIndex = 0;
        self.dataMatchesFound = 0;
        for req in self.reqArray:
            req.reset();
    def processResult(self, line, resultStream, rslt):
        if (rslt != Result.failedMatch):
            print('Processing['+str(self.reqIndex)+']: '+str(rslt));
        if (rslt == Result.failedMatch):
            self.reset();
            self.reqIndex = 0;
        elif (rslt == Result.matchedProgress):
            self.reqIndex += 1
            if (self.reqIndex >= len(self.reqArray)):
                print('ERROR - Last element of RequirementSet cannot return Result.matchedProgress')
                self.reset()
                self.reqIndex = 0
        elif (rslt == Result.oneOfManyMatches):
            #Just keep reading until done
            self.dataMatchesFound += 1
        elif (rslt == Result.endOfSet):
            self.reqIndex += 1
            self.testline(line, resultStream)
        elif (rslt == Result.gameSetMatch):
            print('Game set match!')
            for req in self.reqArray:
                req.printResults(resultStream)
        else:
            self.reqIndex = 0
            cnt = self.indexErrorDic.get(self.reqIndex)
            if (cnt is None):
                cnt = 0;
                print('ERROR - Unknown result type from requirement index: '+str(self.reqIndex))
            cnt += 1;
            self.indexErrorDic[self.reqIndex] = cnt;

    def testline(self, line, resultStream):
        if (self.reqIndex >= len(self.reqArray)):
            print('ERROR - RequirementIndex out of range!')
            reset();
            self.reqIndex = 0;
            resultStream.delete()
            resultStream.close()
        else:
            result = self.reqArray[self.reqIndex].testline(line)
            self.processResult(line, resultStream, self.reqArray[self.reqIndex].testline(line))
                        
                

fileList = []

reqSet = RequirementSet()
addFiles(fileList, '/home/matt/Projects/Web3HackerNetwork/data/github');
print('Found '+str(len(fileList))+' files');
root = '/home/matt/Projects/Web3HackerNetwork/data/samples'
log = open(root+'/commitScan.log', 'w')
for fileClass in fileList:
    linecount = 0
    
    if (fileClass.fileName.endswith('.tsv')):
        print('Skipping '+fileClass.fileName)
    else:
        fq = fileClass.fullyQualified
        tsvFileName = str(fq)+'.tsv'
        result = open(tsvFileName, 'w')

        with open(str(fileClass.fullyQualified), 'r') as file:
            log.write('Reading file: '+fileClass.fileName+'\n')

            for line in file:
                linecount += 1
                reqSet.testline(line, result);
            file.close()
        nBytes = result.tell()
        result.close();
        if (nBytes < 1):
            os.remove(tsvFileName)


        print(str(fileClass.fileName)+' : '+str(linecount)+' lines')
print('All done!')


Found 92 files
unique_project_paths.txt : 40 lines
commit_project_paths.txt : 543 lines
swiss_army_knife.pl : 77 lines
Skipping commit-stat.log.tsv
Processing[0]: Result.matchedProgress
Processing[1]: Result.matchedProgress
Processing[2]: Result.matchedProgress
Processing[3]: Result.matchedProgress
Processing[4]: Result.matchedProgress
Processing[5]: Result.matchedProgress
Processing[6]: Result.endOfSet
Testing for Summary line ( README.md | 2 +-
)
ERROR - Summary line should have comma-separated change and insertion totals: README.md | 2 +-

Testing for Summary line ( README.md | 2 +-
)
ERROR - Summary line should have comma-separated change and insertion totals: README.md | 2 +-

Processing[7]: Result.gameSetMatch
Game set match!


AttributeError: 'str' object has no attribute 'extension'